In [2]:
!pip install tensorflow numpy pillow nltk
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.models import Sequential
import nltk
nltk.download('punkt')
def extract_features(filename):
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    img = load_img(filename, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    features = model.predict(img)
    return features
image_path = '/content/flower.jpg'
features = extract_features(image_path)
print("Extracted features:", features.shape)

captions = ["A flower is beautifull"]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions)
vocab_size = len(tokenizer.word_index) + 1
sequences = []
for caption in captions:
    token_list = tokenizer.texts_to_sequences([caption])[0]
    for i in range(1, len(token_list)):
        seq = token_list[:i+1]
        sequences.append(seq)

max_length = max(len(x) for x in sequences)
sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
X, y = sequences[:, :-1], sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length-1))
model.add(LSTM(256))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
def generate_caption(model, photo, tokenizer, max_length):
    in_text = 'startseq'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length-1, padding='post')
        yhat = model.predict(sequence, verbose=0)
        yhat = np.argmax(yhat)
        word = ''
        for w, index in tokenizer.word_index.items():
            if index == yhat:
                word = w
                break
        if word == 'endseq':
            break
        in_text += ' ' + word
    return in_text


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 23s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Extracted features: (1, 4096)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
